<a href="https://colab.research.google.com/github/vzinkovski/financial-data-science/blob/main/commodities_mean_reversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Explanatory variables:
- Log returns of price ratio
- Volume, OI for each leg
- Call and put options volume, OI for each
- GSCI
- S&P500
- VIX
- WTI, Brent
- USD/AUD, USD/EUR, USD/CAD, USD/BRL

*Note: for all above variables their $n$ lagged values may be considered*